In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("./Resources/lending_data.csv")

df = pd.read_csv(file_path)
# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df['loan_status']

# Separate the X variable, the features
X = df.drop(columns='loan_status')

In [4]:
# Review the y variable Series
y[:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Display the shape of the training and test sets
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((58152, 7), (58152,), (19384, 7), (19384,))

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [9]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [10]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Prediction,Actual
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [11]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, predictions)

0.9520479254722232

In [12]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18663,   102],
       [   56,   563]], dtype=int64)

In [13]:
# Print the classification report for the model
testing_report = classification_report(y_test, predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model had an accuracy score of 99% and a balanced accuracy score Similarly, the FPR for the model prediciting '1' was 9.0% (56 / 619). the overall results may change if more data was available for high-risk loans (and this imbalance will be addressed in the subsequent codes

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [14]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = random_oversampler.fit_resample(X, y)

In [15]:
# Count the distinct values of the resampled labels data
# Convert y_resampled to a pandas Series
y_resampled_series = pd.Series(y_resampled)

# Count distinct values in y_resampled
distinct_counts = y_resampled_series.value_counts()

print("Distinct value counts:")
print(distinct_counts)

Distinct value counts:
1    75036
0    75036
Name: loan_status, dtype: int64


In [17]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_resampled_train, X_resampled_test, y_resampled_train, y_resampled_test = train_test_split(X_resampled, y_resampled, random_state=1)

X_resampled_train.shape, y_resampled_train.shape, X_resampled_test.shape, y_resampled_test.shape

((112554, 7), (112554,), (37518, 7), (37518,))

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [18]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model

classifier2 = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

# Fit the model using the resampled training data
classifier2.fit(X_resampled_train, y_resampled_train)

LogisticRegression(max_iter=200, random_state=1)

In [20]:
# Make predictions using the testing data
predictions_resampled = classifier2.predict(X_resampled_test)
results_resampled = pd.DataFrame({"Prediction": predictions_resampled, "Actual": y_resampled_test})

# print first 5 rows of dataframe
results_resampled.head()

,Prediction,Actual
31735,0,0
85950,1,1
114137,1,1
32165,0,0
26504,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [21]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_resampled_test, predictions_resampled)

0.9950160462226244

In [25]:
# Generate a confusion matrix for the model
confusion_matrix = confusion_matrix(y_resampled_test, predictions_resampled)
confusion_matrix_df = pd.DataFrame(
    confusion_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)
confusion_matrix_df

,Predicted 0,Predicted 1
Actual 0,18810,95
Actual 1,92,18521


In [26]:
# Print the classification report for the model
print(classification_report(y_resampled_test, predictions_resampled))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18905
           1       0.99      1.00      0.99     18613

    accuracy                           1.00     37518
   macro avg       1.00      1.00      1.00     37518
weighted avg       1.00      1.00      1.00     37518



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The precision for the logistic regression model, fit with oversampled data, was great at detecting healthy loans (1.0) and a little less accurate at detecting high-risk one (0.99). High-risk loans improved with a lower FPR of 0.5% when comparing to orginal data.